In [2]:
#!pip install pyarrow
#!pip install tensorflow

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset,TensorDataset
import pandas as pd
import pyarrow
from torchvision import models, transforms
from sklearn.model_selection import train_test_split

In [4]:
#CONSTANTS
exercises_dict = {
    'Abduction': 0,
    'Bird': 1,
    'Bridge': 2,
    'Knee': 3,
    'Shoulder': 4,
    'Squat' : 5,
    'Stretch' : 6
}

In [23]:
dataset = pd.read_parquet('dataset.parquet', engine='pyarrow')

In [24]:
dataset.shape

(2210290, 104)

In [6]:
dataset = dataset.dropna()
#size = 100000
#dataset = dataset[:size]
data = dataset.values
indices = dataset.index
labels = dataset.columns

x_train = data[:,4:]
y_train = np.array([exercises_dict[e] for e in data[:,1]])#[::4])

In [19]:
x_train = x_train.astype('float32')

In [20]:
dataset.shape

(2183099, 104)

In [22]:
#example of a row with labels
print(np.stack((labels.to_numpy(),data[4 * 1793 + 1]), axis = 1))

[['Participant' 'P04']
 ['Exercise' 'Abduction']
 ['Set' 'A']
 ['Camera' 'Frontal_Top']
 ['time(s)' 0.733]
 ['left_ankle_x' 0.013897059485316277]
 ['left_ankle_y' 0.7337374687194824]
 ['left_ankle_z' 0.3085203766822815]
 ['left_ear_x' -0.004014772828668356]
 ['left_ear_y' -0.6559332609176636]
 ['left_ear_z' 0.013528380542993546]
 ['left_elbow_x' 0.2160586714744568]
 ['left_elbow_y' -0.45321035385131836]
 ['left_elbow_z' 0.20046469569206238]
 ['left_eye_center_x' 0.0743502825498581]
 ['left_eye_center_y' -0.6516526341438293]
 ['left_eye_center_z' -0.07846864312887192]
 ['left_eye_inner_x' 0.07402841001749039]
 ['left_eye_inner_y' -0.6512871980667114]
 ['left_eye_inner_z' -0.07923341542482376]
 ['left_eye_outer_x' 0.07386316359043121]
 ['left_eye_outer_y' -0.6522524952888489]
 ['left_eye_outer_z' -0.07846874743700027]
 ['left_foot_x' 0.12216062843799591]
 ['left_foot_y' 0.8188711404800415]
 ['left_foot_z' 0.2560451626777649]
 ['left_heel_x' 0.024564087390899658]
 ['left_heel_y' 0.7773051

In [7]:
#Splits the dataset in a 4-channel data set (think of an RGB Image) coresponding to the 4 different cameras

lengths = np.append(np.where(indices.array[:len(indices)] == 0)[0], [len(indices)])
ft_x, fl_x, st_x, sl_x = [], [], [], []
mod4 = 0
gap = 0

for idx, l in enumerate(lengths):
    if idx < len(lengths)-1:
        end = lengths[idx + 1]
        
        if mod4 == 0:
            gap = end - l

        segment = x_train[l:l + gap]
        
        if mod4 == 0:
            ft_x.append(segment)
        elif mod4 == 1:
            fl_x.append(segment)

        elif mod4 == 2:
            st_x.append(segment)

        elif mod4 == 3:
            sl_x.append(segment)
        
        mod4 = (mod4 + 1) % 4

print("STACKING:")
# Concatenate the lists into arrays
ft_x = np.vstack(ft_x) 
fl_x = np.vstack(fl_x)
st_x = np.vstack(st_x) 
sl_x = np.vstack(sl_x)

x_train = np.array([ft_x, fl_x, st_x, sl_x])
print(ft_x.shape)
print(fl_x.shape)
print(st_x.shape)
print(sl_x.shape)

STACKING:
(552545, 100)
(552545, 100)
(552545, 100)
(552545, 100)


In [8]:
x_train = np.moveaxis(x_train,-1,0)
x_train = np.moveaxis(x_train,-1,0)
y_train = y_train#[:552545]
print(x_train.shape)
print(y_train.shape)

(552545, 100, 4)
(2183099,)


In [9]:
get_mlp = lambda: torch.nn.Sequential(
    torch.nn.Flatten(),
    torch.nn.Linear(400, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 7),
)

get_cnn = lambda: torch.nn.Sequential(
    torch.nn.Conv1d(in_channels=4, out_channels=32, kernel_size=5, stride=1, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(32, 64, 5, stride=2, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(64, 64, 5, stride=1, padding=2),
    torch.nn.ReLU(),
    torch.nn.Conv1d(64, 128, 5, stride=2, padding=2),
    torch.nn.ReLU(),
    torch.nn.AdaptiveAvgPool1d(1),
    torch.nn.Conv1d(128, 7, 1),
    torch.nn.Flatten(),
)

In [10]:

"""
Return a modified dataset cls that can insert MNIST like images into larger
frames with an option for random shifts, scrambling the images in a
consistent way (using the same shuffling for all images) and adding random
Gaussian noise (to the base data, noise is always the same for a given
image).
"""

class CustomDataset(Dataset):
    def __init__(self, x_train, y_train, transform=None):
        self.x_train = x_train
        self.y_train = y_train
        self.transform = transform

    def __len__(self):
        return len(self.x_train)

    def __getitem__(self, idx):
        sample = {'input': self.x_train[idx], 'label': self.y_train[idx]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [51]:
input_size = 400  # Number of input channels
hidden_size = 64  # Size of the hidden layer
output_size = 7  # Number of classes (0 to 6)
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(x_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_mlp()
model_path = 'MLP.path'
model.load_state_dict(torch.load(model_path))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [52]:
#def train_epoch()
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']

        # Flatten the inputs (assuming the data is in the shape (batch_size, 100, 4))
        inputs = inputs.view(-1, input_size)
        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')

Epoch [1/10], Loss: 0.1326
Epoch [2/10], Loss: 0.1248
Epoch [3/10], Loss: 0.1189
Epoch [4/10], Loss: 0.1137
Epoch [5/10], Loss: 0.1089
Epoch [6/10], Loss: 0.1058
Epoch [7/10], Loss: 0.1027
Epoch [8/10], Loss: 0.0990
Epoch [9/10], Loss: 0.0971
Epoch [10/10], Loss: 0.0959


In [53]:
torch.save(model.state_dict(),'MLP.path')

In [14]:
x_train = np.moveaxis(x_train,-1,0)
print(x_train.shape)


(100, 4, 552545)


In [15]:
input_size = 400  # Number of input channels
hidden_size = 64  # Size of the hidden layer
output_size = 7  # Number of classes (0 to 6)
learning_rate = 1e-3
num_epochs = 10

# Create an instance of the custom dataset
custom_dataset = CustomDataset(x_train, y_train)

# Create a DataLoader
batch_size = 64
custom_dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

model = get_cnn()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for epoch in range(num_epochs):
    losses = []
    for batch in custom_dataloader:
        inputs, labels = batch['input'], batch['label']

        # Forward pass
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        print(loss)
        losses.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {np.mean(losses):.4f}')